In [642]:
import cirq

def isolate_first_gate_after_cnot(moments):
    new_moments = moments[:]
    i = 0
    while i < len(new_moments):
        moment = new_moments[i]
        cnot_ops = [op for op in moment.operations if isinstance(op.gate, cirq.CNotPowGate)]
        for cnot in cnot_ops:
            cnot_qubits = set(cnot.qubits)
            j = i + 1
            while j < len(new_moments):
                next_moment = new_moments[j]
                overlapping_ops = [op for op in next_moment.operations if not set(op.qubits).isdisjoint(cnot_qubits)]
                if overlapping_ops:
                    op_to_isolate = overlapping_ops[0]
                    if len(next_moment.operations) > 1:
                        remaining_ops = [op for op in next_moment.operations if op != op_to_isolate]
                        new_moments[j] = cirq.Moment(remaining_ops)
                        new_moments.insert(j, cirq.Moment([op_to_isolate]))
                    break
                j += 1
        i += 1
    return new_moments

def qubits_overlap(ops):
    seen = set()
    for op in ops:
        for q in op.qubits:
            if q in seen:
                return True
            seen.add(q)
    return False

def apply_commutation(circuit: cirq.Circuit) -> cirq.Circuit:
    moments = list(circuit)

    # Step 1: isolate first gate after each CNOT
    moments = isolate_first_gate_after_cnot(moments)

    # Step 2: single pass over adjacent moments to swap commuting ops
    i = 0
    while i < len(moments) - 1:
        m1_ops = list(moments[i].operations)
        m2_ops = list(moments[i + 1].operations)

        to_move_to_m2 = []
        to_move_to_m1 = []

        for op1 in m1_ops:
            for op2 in m2_ops:
                # Commute and overlapping qubits => candidates for swapping
                if cirq.commutes(op1, op2, atol=1e-10) and (not set(op1.qubits).isdisjoint(op2.qubits)):
                    to_move_to_m2.append(op1)
                    to_move_to_m1.append(op2)
                    break

        new_m1_ops = [op for op in m1_ops if op not in to_move_to_m2] + to_move_to_m1
        new_m2_ops = [op for op in m2_ops if op not in to_move_to_m1] + to_move_to_m2

        if not qubits_overlap(new_m1_ops) and not qubits_overlap(new_m2_ops):
            moments[i] = cirq.Moment(new_m1_ops)
            moments[i + 1] = cirq.Moment(new_m2_ops)

        i += 1

    return cirq.Circuit(moments)


In [643]:
import numpy as np
def generate_random_superconducting_circuit(n_qubits, depth):
# Verified 
    qubits = [cirq.LineQubit(i) for i in range(n_qubits)]
    circuit = cirq.Circuit()
    for _ in range(depth):
        layer = []
        i = 0
        for i in range(n_qubits):
            op_type = np.random.choice(['rx', 'rz', 'cx'])
            if op_type == 'rx':
                theta = np.random.choice([(-7*np.pi/4), (-3*np.pi/2), (-5*np.pi/4), (-np.pi), (-3*np.pi/4), (-np.pi/2), (-np.pi/4), 0,
                                        (7*np.pi/4), (3*np.pi/2), (5*np.pi/4), (np.pi), (3*np.pi/4), (np.pi/2), (np.pi/4)])
                layer.append(cirq.rx(theta)(qubits[i]))
            elif op_type == 'rz':
                phi = np.random.choice([(-7*np.pi/4), (-3*np.pi/2), (-5*np.pi/4), (-np.pi), (-3*np.pi/4), (-np.pi/2), (-np.pi/4), 0,
                                        (7*np.pi/4), (3*np.pi/2), (5*np.pi/4), (np.pi), (3*np.pi/4), (np.pi/2), (np.pi/4)])
                layer.append(cirq.rz(phi)(qubits[i]))
            elif op_type == 'cx' and i < len(qubits) - 1:
                if np.random.rand() < 0.5:
                    layer.append(cirq.CNOT(qubits[i], qubits[i + 1]))
        circuit.append(layer)
    return circuit

In [644]:
import numpy as np
def circuits_equivalent(circ1: cirq.Circuit, circ2: cirq.Circuit):
    U1 = cirq.unitary(circ1)
    U2 = cirq.unitary(circ2)
    diff = U1.conj().T @ U2
    phases = np.angle(np.linalg.eigvals(diff))
    global_phase = np.exp(1j * phases[0])
    return np.allclose(global_phase * U1, U2, atol=1E-8)

In [ ]:
# Example usage
def test_1():
    q0, q1 = cirq.LineQubit.range(2)
    circuit = cirq.Circuit(
        cirq.Z(q0),
        cirq.CNOT(q0, q1),
        cirq.Z(q0),
        cirq.Z(q1),
        cirq.H(q1),
        cirq.CNOT(q0, q1),
        cirq.X(q0),
        cirq.Z(q1),
        cirq.Y(q1)
    )

    print("Original Circuit:")
    print(circuit)
    new_circuit = apply_commutation(circuit)
    print("\nTransformed Circuit:")
    print(new_circuit)
    print(f"Circuit equivalence: {circuits_equivalent(circuit, new_circuit)}")

def test_2():
    q0, q1, q2 = cirq.LineQubit.range(3)
    circuit = cirq.Circuit(
        cirq.Z(q0),
        cirq.CNOT(q0, q1),
        cirq.Z(q0),
        cirq.Z(q1),
        cirq.H(q1),
        cirq.CNOT(q0, q1),
        cirq.X(q0),
        cirq.Z(q1),
        cirq.CNOT(q1, q2),
        cirq.Y(q1)
    )

    print("Original Circuit:")
    print(circuit)
    new_circuit = apply_commutation(circuit)
    print("\nTransformed Circuit:")
    print(new_circuit)
    print(f"Circuit equivalence: {circuits_equivalent(circuit, new_circuit)}")

def test_3():
    circuit = generate_random_superconducting_circuit(10, 50)

    print("Original Circuit:")
    print(circuit)
    new_circuit = apply_commutation(circuit)
    print("\nTransformed Circuit:")
    print(new_circuit)
    print(f"Circuit equivalence: {circuits_equivalent(circuit, new_circuit)}")

if __name__ == "__main__":
    print("############ TEST CASE 1: #############")
    test_1()
    print("############ TEST CASE 2: #############")
    test_2()
    print("############ TEST CASE 3: #############")
    test_3()

############ TEST CASE 1: #############
Original Circuit:
0: ───Z───@───Z───────@───X───────
          │           │
1: ───────X───Z───H───X───Z───Y───

Transformed Circuit:
0: ───@───Z───Z───────────@───X───────────
      │                   │
1: ───X───────────Z───H───X───────Z───Y───
Circuit equivalence: True
############ TEST CASE 2: #############
Original Circuit:
0: ───Z───@───Z───────@───X───────────
          │           │
1: ───────X───Z───H───X───Z───@───Y───
                              │
2: ───────────────────────────X───────

Transformed Circuit:
0: ───@───Z───Z───────────@───X───────────────
      │                   │
1: ───X───────────Z───H───X───────@───Z───Y───
                                  │
2: ───────────────────────────────X───────────
Circuit equivalence: True
############ TEST CASE 3: #############
Original Circuit:
0: ───Rx(1.25π)───Rx(-0.5π)────Rx(-π)───────Rx(0.25π)────Rx(0.75π)────Rx(0.75π)───Rz(0.5π)──────────────────────────────────────────────────────